In [1]:
!where python

C:\Users\usermine\Amine-Jupiter\SESSION6\tft\env_tft\Scripts\python.exe
C:\Users\usermine\AppData\Local\Programs\Python\Python311\python.exe
C:\Users\usermine\AppData\Local\Microsoft\WindowsApps\python.exe


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

In [2]:
# Charger les données
df = pd.read_csv("MetroPT3_imputed_final.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)

In [3]:
# Convertir la colonne 'timestamp' en type datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Définir les colonnes continues et catégoriques pour l'entrainement
continuous_features = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]
categorical_features = ["COMP", "DV_eletric", "Towers"]

# Convertir les colonnes catégoriques en type "category"
for col in categorical_features:
    df[col] = df[col].astype('category')

# Convertir la colonne 'panne' en type category
df['panne'] = df['panne'].astype('category')

# Liste des colonnes nécessaires
columns_to_keep = ["timestamp"] + continuous_features + ["panne"] + categorical_features

# Réduire le DataFrame
df = df[columns_to_keep]

# Afficher les premières lignes pour vérifier
display(df.head())

,timestamp,TP2,DV_pressure,Oil_temperature,Motor_current,Reservoirs,panne,COMP,DV_eletric,Towers
0,2020-02-01 00:00:00,-0.012,-0.024,53.600,0.0400,9.358,0,1.0,0.0,1.0
1,2020-02-01 00:00:10,-0.014,-0.022,53.675,0.0400,9.348,0,1.0,0.0,1.0
2,2020-02-01 00:00:20,-0.012,-0.022,53.600,0.0425,9.338,0,1.0,0.0,1.0
3,2020-02-01 00:00:30,-0.012,-0.022,53.425,0.0400,9.328,0,1.0,0.0,1.0
4,2020-02-01 00:00:40,-0.012,-0.022,53.475,0.0400,9.318,0,1.0,0.0,1.0


In [4]:
# Créer la colonne 'time_idx' basée sur les timestamps
df['time_idx'] = ((df['timestamp'] - df['timestamp'].min()).dt.total_seconds() // 10).astype(int)

# Ajouter une colonne 'group_id' pour identifier les groupes (utile pour un seul groupe)
df['group_id'] = "compresseur"
display(df.head())

,timestamp,TP2,DV_pressure,Oil_temperature,Motor_current,Reservoirs,panne,COMP,DV_eletric,Towers,time_idx,group_id
0,2020-02-01 00:00:00,-0.012,-0.024,53.600,0.0400,9.358,0,1.0,0.0,1.0,0,compresseur
1,2020-02-01 00:00:10,-0.014,-0.022,53.675,0.0400,9.348,0,1.0,0.0,1.0,1,compresseur
2,2020-02-01 00:00:20,-0.012,-0.022,53.600,0.0425,9.338,0,1.0,0.0,1.0,2,compresseur
3,2020-02-01 00:00:30,-0.012,-0.022,53.425,0.0400,9.328,0,1.0,0.0,1.0,3,compresseur
4,2020-02-01 00:00:40,-0.012,-0.022,53.475,0.0400,9.318,0,1.0,0.0,1.0,4,compresseur


In [5]:
# Diviser les données en ensembles d'entraînement et de validation
#train_data = df[(df['timestamp'] >= '2020-02-01 00:00:00') & (df['timestamp'] <= '2020-06-07 14:30:00')]
#val_data   = df[(df['timestamp'] >= '2020-06-07 14:30:10') & (df['timestamp'] <= '2020-09-01 03:59:50')]

# Diviser les données en ensembles d'entraînement et de validation
train_data = df[(df['timestamp'] >= '2020-05-29 22:00:00') & (df['timestamp'] <= '2020-05-29 23:45:00')]
val_data   = df[(df['timestamp'] >= '2020-07-15 13:00:00') & (df['timestamp'] <= '2020-07-15 14:45:00')]

# Vérifier les tailles des ensembles
print(f"Entraînement : {len(train_data)} lignes")
print(f"Validation : {len(val_data)} lignes")

# Vérifiez les classes dans le jeu de données d'entraînement et de validation
print("Classes uniques dans le jeu de données d'entraînement :", train_data["panne"].unique())
print("Classes uniques dans le jeu de données de validation :", val_data["panne"].unique())

Entraînement : 631 lignes
Validation : 631 lignes
Classes uniques dans le jeu de données d'entraînement : [0, 2, 1]
Categories (3, int64): [0, 1, 2]
Classes uniques dans le jeu de données de validation : [0, 2, 1]
Categories (3, int64): [0, 1, 2]


In [11]:
for col in categorical_features:
    train_data.loc[:, col] = train_data[col].astype(int).astype(str).astype('category')
    val_data.loc[:, col] = val_data[col].astype(int).astype(str).astype('category')

# Convertir en catégorie en utilisant .loc pour éviter l'avertissement
train_data.loc[:, "panne"] = train_data["panne"].astype(int).astype(str).astype('category')
val_data.loc[:, "panne"] = val_data["panne"].astype(int).astype(str).astype('category')

# Vérifier les types des colonnes
print(df.dtypes)    

timestamp          datetime64[ns]
TP2                       float64
DV_pressure               float64
Oil_temperature           float64
Motor_current             float64
Reservoirs                float64
panne                    category
COMP                     category
DV_eletric               category
Towers                   category
time_idx                    int64
group_id                   object
dtype: object


<ul style="text-align: center;font-family: times, serif; font-size:14pt; color:Red;">
<strong>#########################  Préparation des données pour le modèle ############################</strong>
</ul>

In [ ]:
# Définir les longueurs des séquences

# Créer le dataset d'entraînement
training = TimeSeriesDataSet(
    train_data,
    time_idx="time_idx",
    target="panne",  # Cible définie comme 'panne'
    group_ids=["group_id"],
    min_encoder_length=90,  # Ajuster selon votre besoin
    max_encoder_length=180,
    max_prediction_length=1,
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"],
    time_varying_unknown_categoricals=["COMP", "DV_eletric", "Towers"],
)

# Créer le dataset de validation
validation = TimeSeriesDataSet.from_dataset(training, val_data)

<ul style="text-align: center;font-family: times, serif; font-size:14pt; color:Red;">
<strong>######################### Charger les DataLoaders ############################</strong>
</ul>

In [ ]:
# Créer les DataLoaders pour l'entraînement et la validation
train_dataloader = training.to_dataloader(train=True, batch_size=64, num_workers=4, batch_sampler=None)
val_dataloader = validation.to_dataloader(train=False, batch_size=64, num_workers=4, batch_sampler=None)

<ul style="text-align: center;font-family: times, serif; font-size:14pt; color:Red;">
<strong>######################### Initialiser le modèle TFT ############################</strong>
</ul>

In [ ]:
# Initialiser le modèle TFT
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,     # Taux d'apprentissage pour l'optimisation
    hidden_size=16,         # Taille des couches cachées
    attention_head_size=4,  # Nombre de têtes d'attention
    dropout=0.1,            # Pour éviter le surapprentissage
    hidden_continuous_size=8,  # Taille des représentations continues
    loss=QuantileLoss(),       # Fonction de perte pour les séries temporelles
    log_interval=10,           # Journaliser toutes les 10 itérations
    reduce_on_plateau_patience=4,  # Réduire le learning rate si stagnation
    optimizer="adam",             # Explicitez l'optimiseur
)

print("Modèle initialisé avec succès.")


In [ ]:
# Encapsulation dans LightningModule
class CustomTFT(LightningModule):
    def __init__(self, tft_model):
        super().__init__()
        self.tft_model = tft_model

    def forward(self, x):
        return self.tft_model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        if isinstance(y, tuple):
            y = y[0]
        y_hat = self.tft_model(x)
        predictions = y_hat["prediction"]
        loss = self.tft_model.loss(predictions, y)
        self.log("train_loss", loss, batch_size=y.size(0))
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        if isinstance(y, tuple):
            y = y[0]
        y_hat = self.tft_model(x)
        predictions = y_hat["prediction"]
        val_loss = self.tft_model.loss(predictions, y)
        self.log("val_loss", val_loss, batch_size=y.size(0))
        return val_loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.03)


# Créer un modèle encapsulé
model = CustomTFT(tft)

<ul style="text-align: center;font-family: times, serif; font-size:14pt; color:Red;">
<strong>######################### Entraîner le modèle ############################</strong>
</ul>

In [ ]:
from pytorch_lightning import Trainer

# Ajouter les callbacks pour Early Stopping et sauvegarde du modèle
early_stopping = EarlyStopping(
    monitor="val_loss",  # Surveiller la métrique val_loss
    patience=5,          # Arrêter après 5 époques sans amélioration
    verbose=True,
    mode="min"           # Minimiser la perte
)

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",  # Sauvegarder le meilleur modèle basé sur val_loss
    dirpath="checkpoints",  # Dossier pour les checkpoints
    filename="best-checkpoint",  # Nom du fichier de checkpoint
    save_top_k=1,               # Sauvegarder uniquement le meilleur modèle
    mode="min"                  # Minimiser la perte
)


# Configurer le Trainer
trainer = Trainer(
    max_epochs=2,                # Nombre d'époques maximum
    accelerator="cpu",           # Utilisation du CPU
    gradient_clip_val=0.1,       # Clip pour éviter les gradients explosifs
    callbacks=[early_stopping, checkpoint_callback],  # Ajouter les callbacks
)

# Entraîner le modèle
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

# Vérifier si l'entraînement a été interrompu et sauvegarder l'état
trainer.save_checkpoint("last-phase-checkpoint.ckpt")

<ul style="text-align: center;font-family: times, serif; font-size:14pt; color:Red;">
<strong>######################### Reprendre l'entraînement ############################</strong>
</ul>

In [6]:
print(df.dtypes)   

timestamp          datetime64[ns]
TP2                       float64
DV_pressure               float64
Oil_temperature           float64
Motor_current             float64
Reservoirs                float64
panne                    category
COMP                     category
DV_eletric               category
Towers                   category
time_idx                    int64
group_id                   object
dtype: object


In [9]:
import pandas as pd
import torch
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.metrics import SMAPE, MAE, RMSE
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

# Charger les données
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Convertir les colonnes catégoriques en chaînes de caractères
df['COMP'] = df['COMP'].astype(str)
df['DV_eletric'] = df['DV_eletric'].astype(str)
df['Towers'] = df['Towers'].astype(str)
df['group_id'] = df['group_id'].astype(str)

# Préparation des ensembles d'entraînement et de validation
train_data = df[(df['timestamp'] >= '2020-05-29 22:00:00') & (df['timestamp'] <= '2020-05-29 23:45:00')]
val_data = df[(df['timestamp'] >= '2020-07-15 13:00:00') & (df['timestamp'] <= '2020-07-15 14:45:00')]

# Créer des TimeSeriesDataSet
max_encoder_length = 30  # Longueur de la fenêtre passée
max_prediction_length = 1  # Prévision unique

# Définir le DataSet pour l'entraînement
training = TimeSeriesDataSet(
    train_data,
    time_idx="time_idx",
    target="panne",
    group_ids=["group_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["group_id"],
    time_varying_known_categoricals=["COMP", "DV_eletric", "Towers"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"],
)

# Définir le DataSet pour la validation
validation = TimeSeriesDataSet.from_dataset(training, val_data, predict=True)

# Créer DataLoader pour l'entraînement et la validation
train_dataloader = training.to_dataloader(train=True, batch_size=64, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=64, num_workers=0)

# Initialisation du modèle TFT comme LightningModule
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=1e-3,
    hidden_size=16,
    attention_head_size=4,
    dropout=0.1,
    loss=SMAPE(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

# Vérification explicite que le modèle est compatible
assert isinstance(tft, torch.nn.Module), "Le modèle doit être une instance de LightningModule ou torch.nn.Module."

# Configuration de l'entraînement
early_stop_callback = EarlyStopping(monitor="val_loss", patience=10, verbose=True, mode="min")
trainer = Trainer(
    max_epochs=30,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1 if torch.cuda.is_available() else 1,  # Correction ici
    gradient_clip_val=0.1,
    callbacks=[early_stop_callback],
)

# Entraîner le modèle
trainer.fit(model=tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

# Évaluer les performances
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = tft.predict(val_dataloader)

# Afficher les métriques
smape = SMAPE()(predictions, actuals).item()
mae = MAE()(predictions, actuals).item()
rmse = RMSE()(predictions, actuals).item()

print(f"SMAPE: {smape:.2f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}")


C:\Users\usermine\Amine-Jupiter\SESSION6\tft\env_tft\Lib\site-packages\pytorch_forecasting\data\timeseries.py:1207: UserWarning: If predicting, no randomization should be possible - setting stop_randomization=True
  warnings.warn(
C:\Users\usermine\Amine-Jupiter\SESSION6\tft\env_tft\Lib\site-packages\lightning\pytorch\utilities\parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
C:\Users\usermine\Amine-Jupiter\SESSION6\tft\env_tft\Lib\site-packages\lightning\pytorch\utilities\parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
C:\Users\usermine\Amine-Jupiter\SESSION6\tft\env_tft\Lib\site-packages\pytorch_forecasting\models\temporal_fusion_transformer\__init__.py:171: UserWarning: In pytorch-

TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `TemporalFusionTransformer`

In [10]:
pip list | findstr torch
pip list | findstr lightning
pip list | findstr forecasting

SyntaxError: invalid syntax (4037731253.py, line 1)

<ul style="text-align: center;font-family: times, serif; font-size:14pt; color:Red;">
<strong>######################### Faire des prédictions ############################</strong>
</ul>

In [ ]:
# Faire des prédictions
raw_predictions, x = tft.predict(val_dataloader, mode="raw", return_x=True)

# Afficher les prédictions brutes
print(raw_predictions)

In [ ]:
# Obtenir les prédictions
predictions = tft.predict(val_dataloader)

# Afficher les dimensions des prédictions
print(f"Shape of predictions: {predictions.shape}")


In [ ]:
# Afficher quelques exemples de prédictions
print("Example predictions:", predictions[:5])

<ul style="text-align: center;font-family: times, serif; font-size:14pt; color:Red;">
<strong>######################### Évaluation ############################</strong>
</ul>

In [ ]:
# Les métriques de validation sont enregistrées par le Trainer
print(f"Meilleure perte de validation (val_loss): {trainer.callback_metrics['val_loss']}")


In [ ]:
import torch
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from pytorch_forecasting.metrics import SMAPE, MAE, RMSE

# Charger le modèle depuis le checkpoint si nécessaire
# Si le modèle est déjà en mémoire, cette étape n'est pas requise
# model = CustomTFT.load_from_checkpoint("last-phase-checkpoint.ckpt")

# Préparer le modèle pour l'évaluation
model.eval()

# Faire des prédictions sur l'ensemble de validation
all_preds = []
all_targets = []

with torch.no_grad():
    for batch in val_dataloader:
        x, y = batch

        # Si y est un tuple, extraire la cible réelle
        if isinstance(y, tuple):
            y = y[0]

        # Obtenir les prédictions
        y_hat = model.tft_model(x)["prediction"]

        # Convertir les prédictions en classe la plus probable
        preds = torch.argmax(y_hat, dim=-1).cpu().numpy()
        targets = y.cpu().numpy()

        all_preds.append(preds)
        all_targets.append(targets)

# Convertir les listes en tableaux numpy
all_preds = np.concatenate(all_preds)
all_targets = np.concatenate(all_targets)

# Calculer les métriques par classe
print("=== Rapport de classification par classe ===")
print(classification_report(all_targets, all_preds, zero_division=0))

# Matrice de confusion
print("\n=== Matrice de confusion ===")
print(confusion_matrix(all_targets, all_preds))

# Calculer les métriques globales (SMAPE, MAE, RMSE)
predictions = model.tft_model.predict(val_dataloader)  # Obtenir toutes les prédictions
actuals = torch.cat([y for x, y in iter(val_dataloader)])  # Cibles réelles

# SMAPE, MAE, RMSE
smape = SMAPE()(predictions, actuals).item()
mae = MAE()(predictions, actuals).item()
rmse = RMSE()(predictions, actuals).item()

# Afficher les métriques globales
print("\n=== Métriques globales ===")
print(f"SMAPE: {smape:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")


In [ ]:
# Convertir en catégorie
train_data["panne"] = train_data["panne"].astype("category")
val_data["panne"] = val_data["panne"].astype("category")





In [ ]:
# Vérifier les catégories après nettoyage
print("Catégories dans l'ensemble d'entraînement :", train_data["panne"].cat.categories)
print("Catégories dans l'ensemble de validation :", val_data["panne"].cat.categories)

In [ ]:
print("Classes uniques dans la variable 'panne' :")
print(df["panne"].unique())



In [ ]:
print("Classes uniques dans l'ensemble d'entraînement :")
print(train_data["panne"].unique())

print("Classes uniques dans l'ensemble de validation :")
print(val_data["panne"].unique())


In [ ]:
print("Distribution des classes dans l'ensemble d'entraînement :")
print(train_data["panne"].value_counts())

print("Distribution des classes dans l'ensemble de validation :")
print(val_data["panne"].value_counts())


In [ ]:
train_data["panne"] = train_data["panne"].cat.remove_unused_categories()
val_data["panne"] = val_data["panne"].cat.remove_unused_categories()


In [ ]:
print("Catégories définies :", validation.categoricals)
print("Reels définis :", validation.reals)


In [ ]:
# Vérifier les classes uniques dans la colonne cible
print("Classes uniques dans la colonne 'panne' :")
print(validation.data["panne"].unique())


In [ ]:
# Matrice de confusion
print("Matrice de confusion :")
print(predictions)

<ul style="text-align: center;font-family: times, serif; font-size:14pt; color:Red;">
<strong>######################### Sauvegarder le modèle ############################</strong>
</ul>

In [ ]:
# Sauvegarder le modèle
model_checkpoint = "tft_model_1_2025_01_02_08_20_epoch2.ckpt"
trainer.save_checkpoint(model_checkpoint)
print(f"Modèle sauvegardé sous : {model_checkpoint}")


In [ ]:
import torch

# Sauvegarder uniquement les poids du modèle
torch.save(tft.state_dict(), "tft_model_weights_2025_01_02_08_20_epoch2.pth")
print("Poids du modèle sauvegardés avec succès !")


In [ ]:
import pickle
from pytorch_forecasting import TemporalFusionTransformer

# Charger ou initialiser votre modèle TFT
tft = TemporalFusionTransformer(...)  # Assurez-vous que le modèle est déjà entraîné

# Sauvegarder le modèle en fichier `.pkl`
with open("tft_model.pkl", "wb") as f:
    pickle.dump(tft, f)

print("Modèle sauvegardé en 'tft_model.pkl'")


In [ ]:
from pytorch_forecasting import TemporalFusionTransformer

# Charger le modèle depuis le checkpoint
tft = TemporalFusionTransformer.load_from_checkpoint("tft_model_final.ckpt")
print("Modèle chargé avec succès !")
